In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import cv2
import os
import pickle
import os.path as path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from keras import Model, Input
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from keras.layers import Dense,BatchNormalization,Activation
from keras.layers import Dropout
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

In [2]:
def logisticregression(X_train,y_train,X_test,y_test):
    c1 = LogisticRegression(random_state=0)
    c1.fit(X_train, y_train)
    accuracy = c1.score(X_test, y_test)
    return accuracy*100

def knn(X_train,y_train,X_test,y_test,nneighbors=10):
    c2 = KNeighborsClassifier(n_neighbors=nneighbors, metric='minkowski', p=2)
    c2.fit(X_train, y_train)
    accuracy = c2.score(X_test, y_test)
    return accuracy*100

def svm(X_train,y_train,X_test,y_test):
    c3 = SVC(kernel='linear', random_state=0)
    c3.fit(X_train, y_train)
    accuracy = c3.score(X_test, y_test)
    return accuracy*100

def naivebayes(X_train,y_train,X_test,y_test):
    c6 = GaussianNB()
    c6.fit(X_train, y_train)
    accuracy = c6.score(X_test, y_test)
    return accuracy*100

def decisiontree(X_train,y_train,X_test,y_test):
    c7 = DecisionTreeClassifier(criterion = 'entropy',random_state=0,max_depth = 6)
    c7.fit(X_train, y_train)
    accuracy = c7.score(X_test, y_test)
    return accuracy*100

def randomforest(X_train,y_train,X_test,y_test,nestimators=10):
    c8 = RandomForestClassifier(n_estimators=20, random_state=12,max_depth=5)
    c8.fit(X_train, y_train)
    accuracy = c8.score(X_test, y_test)
    return accuracy*100

In [5]:
X=np.load('/content/drive/MyDrive/Face Recognition using ML/bigdata_last_submission.npy')
y=np.load('/content/drive/MyDrive/Face Recognition using ML/bigtarget_last_submisison.npy')

In [52]:
print(X.shape)
X=np.reshape(X,(X.shape[0],-1))
print(X.shape)
print(y.shape)

(3240, 90000)
(3240, 90000)
(3240,)


In [61]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [62]:
f=open('/content/drive/MyDrive/Face Recognition using ML/latest_store.pckl', 'rb')
labels_to_names=pickle.load(f)
f.close()

In [63]:
labels_to_names

{0: 'Robert_Downey_Jr',
 1: 'Peri_Gilpin',
 2: 'Lisa_Kudrow',
 3: 'Ryan_Reynolds',
 4: 'Oliver_Platt',
 5: 'Seth_Rogen',
 6: 'Kristen_Johnston',
 7: 'Melissa_Fumero',
 8: 'Chris_Rock',
 9: 'Alec_Baldwin',
 10: 'Annie_Ilonzeh',
 11: 'Chris_Evans',
 12: 'Danny_Glover'}

In [64]:
pca = PCA(n_components=50)
pca.fit(X_train)
X_train=pca.transform(X_train)
X_test=pca.transform(X_test)

In [65]:
print(X_train.shape)

(2430, 50)


In [66]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [67]:
acc_log=logisticregression(X_train,y_train,X_test,y_test)
acc_knn=knn(X_train,y_train,X_test,y_test)
acc_svm=svm(X_train,y_train,X_test,y_test)
acc_naive=naivebayes(X_train,y_train,X_test,y_test)
acc_dectree=decisiontree(X_train,y_train,X_test,y_test)
acc_randforest=randomforest(X_train,y_train,X_test,y_test)

In [68]:
models = pd.DataFrame({
                        'Model': ['Logistic Regression','KNN','SVM','Naive Bayes',
                                  'Decision Tree','Random Forest'],
                        'Score': [acc_log, acc_knn, acc_svm, acc_naive,
                                  acc_dectree, acc_randforest]
                    })

print(models.sort_values(by='Score', ascending=False))

                 Model      Score
2                  SVM  97.407407
0  Logistic Regression  84.320988
1                  KNN  74.197531
3          Naive Bayes  73.333333
5        Random Forest  73.086420
4        Decision Tree  48.518519


In [22]:
X_train.shape

(2430, 50)

In [23]:
X_test.shape

(810, 50)

In [24]:
#mod=MLPClassifier(solver = 'lbfgs', alpha = 1e-5,hidden_layer_sizes = (3500,), random_state = 1)
mod=LogisticRegression(random_state=0)

In [25]:
mod.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
val=mod.predict(X_test)
ser=pd.Series(val)

In [27]:
sum(val!=y_test)

299

In [28]:
f=open('/content/drive/MyDrive/Face Recognition using ML/latest_target.pckl', 'rb')
real_target=pickle.load(f)
f.close()

In [29]:
# mod=SVC(kernel='rbf', C=2)
# mod.fit(X_train, y_train)
# mod.score(X_train, y_train)

### TESTING WITHOUT EXTRACTING FACE

In [30]:
def new_data_preprocessing(img,pca,scaler,img_size):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)      
    resized=cv2.resize(gray,(img_size,img_size))
    resized=resized/255.
    resized=np.reshape(resized,(1,-1))
    img2=pca.transform(resized)
    img2=scaler.transform(img2)
    print(img2.shape)
    #resized=np.reshape(resized,(1,img_size,img_size,1))
    #img=np.reshape(resized,(1,-1))
    return img2

In [31]:
img_size=300
filename=os.path.join("newtest/00.jpg")
img=cv2.imread(filename)
img_bgr=img.copy()
imgf=new_data_preprocessing(img,pca,scaler,img_size)
predict_label=mod.predict(imgf)
print(labels_to_names[predict_label[0]])

img_bgr=cv2.resize(img_bgr,(img_size,img_size))
img_rgb=cv2.cvtColor(img_bgr,cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)

AttributeError: ignored

# .

In [ ]:
labels_to_names

In [ ]:
def detect_all_face(img):
    face_img=img.copy()
    face_rects=face_cascade.detectMultiScale(face_img) 
    count=0
    for (x,y,w,h) in face_rects: 
        cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,255,255),10)  
        cv2.putText(face_img, str(count), (x, y),cv2.FONT_HERSHEY_SIMPLEX,6,(0,0,255),10)
        count+=1
    return face_img

In [ ]:
### TESTING WITH EXTRACTING FACE - 00 perfect example
def detect_face(colour_image,mod,pca,scaler,face_cascade,img_size): #take BGR colour image
    boxes_list=[]
    probability_list=[]     
#     gray=cv2.resize(colour_image,(img_size,img_size))
    gray=colour_image.copy()
    img=cv2.cvtColor(gray,cv2.COLOR_BGR2GRAY) 
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img) 
    for (x,y,w,h) in face_rects: 
        #cv2.rectangle(face_img, (x,y), (x+w,y+h), (255,255,255), 2) 
        i2=face_img.copy()
        i2=i2[y:y+h,x:x+w]
        resized=cv2.resize(i2,(img_size,img_size))
        resized=resized/255.
        resized=np.reshape(resized,(1,-1))
        img=pca.transform(resized)
        img=scaler.transform(img)
        
        probability=mod.predict_proba(img)
        probability=probability[0] #1* array
        
        boxes_list.append((x,y,w,h))
        probability_list.append(probability)
#         plt.imshow(i2,cmap='gray')
        
    #print(probability_list)
    probability_list=np.array(probability_list)
    print("&"*20)
    print(probability_list)
    
    
#     index=np.where(probability_list==probability_list.max())[0][0]
    
#     prediction_label=np.where(probability_list==probability_list.max())[1][0]
    
#     x,y,w,h=boxes_list[index]
#     cv2.rectangle(face_img, (x,y), (x+w,y+h), (255,0,0), 2)
#     return face_img,prediction_label,(x,y,w,h)
    
    #ALL IMAGES(START)
    
    c1=0
    preds=[]
    coords=[]
    print("@"*20)
    for i in probability_list:
        prediction_label=np.where(i==i.max())[0][0]
        second_highest_label=np.where(i==sorted(i)[-2])[0][0]
        print(i.max()-sorted(i)[-2])
        if(i.max()-sorted(i)[-2]<0.3):
            c1+=1
            continue
        preds.append(prediction_label)
        x,y,w,h=boxes_list[c1]
        coords.append((x,y,w,h))
        cv2.rectangle(face_img, (x,y), (x+w,y+h), (255,0,0), 2)
        c1+=1
    return face_img,preds,coords,boxes_list
    #ALL IMAGES(END)
    
    
#     face_img=cv2.resize(face_img,(img_size,img_size))

In [ ]:
img_size=300
face_cascade=cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

# filename=os.path.join("newtest/rock_test1.jpg") #rock_test1
filename=os.path.join("mixtest/9.jpg") #rock_test1

img=cv2.imread(filename)

all_faces_image=img.copy()
# all_faces_image=cv2.resize(all_faces_image,(img_size,img_size))
all_faces_image=cv2.cvtColor(all_faces_image,cv2.COLOR_BGR2RGB)
# all_faces_image=detect_all_face(all_faces_image)
# temp=all_faces_image.copy()
# all_faces_image=cv2.resize(all_faces_image,(img_size,img_size))

gray_image,prediction_label,box_coordinates,all_cords=detect_face(img,mod,pca,scaler,face_cascade,img_size)
#print(str(prediction_label)+"------->"+str(labels_to_names[prediction_label]))

#--------------------------------------------------------------

count=0
for (x,y,w,h) in all_cords: 
    cv2.rectangle(all_faces_image,(x,y),(x+w,y+h),(255,255,255),10)  
    cv2.putText(all_faces_image, str(count), (x, y),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,255),6)
    count+=1
all_faces_image=cv2.resize(all_faces_image,(img_size,img_size))

#--------------------------------------------------------------

b=cv2.imread(filename)
colour_img=cv2.cvtColor(b,cv2.COLOR_BGR2RGB)
plain_colour_image=colour_img.copy()
for i in range(len(box_coordinates)):
    x,y,w,h=box_coordinates[i]
    cv2.rectangle(colour_img, (x,y), (x+w,y+h), (0,0,255), 5)
    cv2.putText(colour_img, labels_to_names[prediction_label[i]], (x, y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),5)

colour_img=cv2.resize(colour_img,(img_size,img_size))
plain_colour_image=cv2.resize(plain_colour_image,(img_size,img_size))  

clear_output(wait=True)
#------------------------------------------------------------
# x,y,w,h=box_coordinates
# b=cv2.imread(filename)
# # b=cv2.resize(b,(img_size,img_size))
# colour_img=cv2.cvtColor(b,cv2.COLOR_BGR2RGB)
# plain_colour_image=colour_img.copy()
# cv2.rectangle(colour_img, (x,y), (x+w,y+h), (0,0,255), 5)
# # cv2.rectangle(colour_img,(x,y-40),(x+w,y),(255,0,0),-1)
# cv2.putText(colour_img, labels_to_names[prediction_label], (x, y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),5)
# colour_img=cv2.resize(colour_img,(img_size,img_size))
# plain_colour_image=cv2.resize(plain_colour_image,(img_size,img_size))
# # plt.imshow(colour_img)

# #plt.imshow(gray_image,cmap='gray')


# clear_output(wait=True)

# #print(str(prediction_label)+"------->"+str(labels_to_names[prediction_label]))
# #plt.imshow(all_faces_image)
# #plt.imshow(colour_img)

In [ ]:
plt.imshow(all_faces_image)
plt.plot()

In [ ]:
fig=plt.figure(figsize=(16,16))
plt.subplot(1,3,1)
plt.imshow(plain_colour_image)
plt.subplot(1,3,2)
plt.imshow(colour_img)
plt.show()

# For All Trained

In [ ]:
f=open('latest_filenames.pckl', 'rb')
fnames=pickle.load(f)
f.close()

In [ ]:
new_prediction_result=[]
for path in fnames:
    img=cv2.imread(path)
    img_bgr=img.copy()
    img=new_data_preprocessing(img,pca,scaler,img_size)
    predict_label=mod.predict(img)

    img_bgr=cv2.resize(img_bgr,(img_size,img_size))
    img_rgb=cv2.cvtColor(img_bgr,cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.show()
    new_prediction_result.append(predict_label[0])
    print(labels_to_names[predict_label[0]])

In [ ]:
# sum(np.array(new_prediction_result)==real_target)/len(real_target)